# Deliverable 2: Determine Bias of Vine Reviews

In [1]:
import os
# Find the latest version of spark 3.2.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Hi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2022-11-03 01:21:53--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar’

postgresql-42.5.0.j 100%[===================>]   1022K  5.08MB/s    in 0.2s    

2022-11-03 01:21:53 (5.08 MB/s) - ‘postgresql-42.5.0.jar’ saved [1046274/1046274]



In [3]:
# Create Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Deliverable2").config("spark.driver.extraClassPath","/content/postgresql-42.5.0.jar").getOrCreate()

In [4]:
# Load Amazon Reviews into Spark Dataframe
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [5]:
# Create the vine_table DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [6]:
# Filter dataframe for helpful reviews (total_votes >= 20)
filtered_reviews_df = vine_df.filter("total_votes>=20")
filtered_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [7]:
# View data types
filtered_reviews_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [8]:
# Change "helpful_votes" and "total_votes" columns to floats
from pyspark.sql.types import FloatType
filtered_reviews_df = filtered_reviews_df.withColumn('helpful_votes', filtered_reviews_df['helpful_votes'].cast(FloatType()))
filtered_reviews_df = filtered_reviews_df.withColumn('total_votes', filtered_reviews_df['total_votes'].cast(FloatType()))

In [9]:
# Select only rows where the % of helpful_votes/total_votes >= 50%)
helpful_reviews_df = filtered_reviews_df.filter("helpful_votes/total_votes >= .50")
helpful_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|         30.0|       32.0|   N|                N|
| RQW4AFOG9MR4Z|          5|         51.0|       52.0|   N|                N|
|R2SMUEBMGLAJQK|          5|         29.0|       36.0|   N|                Y|
|R1XVIZZALU5P6J|          5|        148.0|      150.0|   N|                Y|
|R11UFMG8M2488I|          4|         23.0|       26.0|   N|                N|
|R2OSHKSPXU68W5|          5|        136.0|      147.0|   N|                N|
|R3M02FSD3BLUPU|          5|         19.0|       21.0|   N|                N|
|R1P0LEEJHH09L3|          5|         32.0|       32.0|   N|                Y|
|R1V6UU1EHW3Q12|          5|         57.0|       62.0|   N|                Y|
|R230W87COFSTCL|          5|         21.0|       21.0|   N|     

In [10]:
# Filter for reviews made by members of the Vine Program 
helpful_vine_reviews_df = helpful_reviews_df.filter(helpful_reviews_df.vine == 'Y')
helpful_vine_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6U9701C3BGO6|          3|        139.0|      147.0|   Y|                N|
|R1XK3ALB45D7N4|          5|         33.0|       34.0|   Y|                N|
|R1IZCSTLX81D6C|          5|         31.0|       33.0|   Y|                N|
|R2C8NC8EQLH4JF|          3|         45.0|       48.0|   Y|                N|
|R1JJ1YOJMOML1P|          5|         18.0|       21.0|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
# Filter for reviews not made by members of the Vine Program
not_vine_reviews_df = helpful_reviews_df.filter(helpful_reviews_df.vine == 'N')
not_vine_reviews_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|         30.0|       32.0|   N|                N|
| RQW4AFOG9MR4Z|          5|         51.0|       52.0|   N|                N|
|R2SMUEBMGLAJQK|          5|         29.0|       36.0|   N|                Y|
|R1XVIZZALU5P6J|          5|        148.0|      150.0|   N|                Y|
|R11UFMG8M2488I|          4|         23.0|       26.0|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



## Create Calculations to Determine Bias

In [12]:
# Create calculatios to determine possible bias using count(). Separate into paid and unpaid
from pyspark.sql.functions import count

# Total Number of Reviews (Paid vs Unpaid)
paid_reviews_total = helpful_vine_reviews_df.count()
unpaid_reviews_total = not_vine_reviews_df.count()

print("Number of helpful, paid Apparel Reviews: %f" % paid_reviews_total)
print("Number of helpful, unpaid Apparel Reviews: %f" % unpaid_reviews_total)

Number of helpful, paid Apparel Reviews: 33.000000
Number of helpful, unpaid Apparel Reviews: 45388.000000


In [13]:
# Calclate the number of 5-star reviews, Separate into paid and unpaid
paid_5star_reviews = helpful_vine_reviews_df.filter(helpful_vine_reviews_df['star_rating'] == 5)
unpaid_5star_reviews = not_vine_reviews_df.filter(not_vine_reviews_df['star_rating'] == 5)

print("Number of 5-star helpful, PAID Apparel Reviews: %f" % paid_5star_reviews.count())
print(" Number of 5-star helpful UNPAID Apparel Reviews: %f" % unpaid_5star_reviews.count())

Number of 5-star helpful, PAID Apparel Reviews: 15.000000
 Number of 5-star helpful UNPAID Apparel Reviews: 23733.000000


In [15]:
# Calculate the percentage of 5-star reviews. Separate into paid and unpaid
paid_5star_review_count = paid_5star_reviews.count()
unpaid_5star_review_count = unpaid_5star_reviews.count()

paid_5star_percentage = (paid_5star_review_count/paid_reviews_total) * 100
unpaid_5star_percentage = (unpaid_5star_review_count/unpaid_reviews_total) * 100
print("Percentage of  5-star,  PAID Apparel Reviews: %f" % paid_5star_percentage)
print(" Number of Helpful, 5-star UNPAID Apparel Reviews: %f" % unpaid_5star_percentage)

Percentage of  5-star,  PAID Apparel Reviews: 45.454545
 Number of Helpful, 5-star UNPAID Apparel Reviews: 52.289151
